In [ ]:
import numpy as np
import pandas as pd
import networkx as nx # https://networkx.org/documentation/stable/index.html

In [ ]:
# read in meta data
metadata = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", nrows=1, header=None).iloc[0]
s = metadata.iloc[0]
n = metadata.iloc[1]
m = metadata.iloc[2]
l = metadata.iloc[3]

In [ ]:
df = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", skiprows=1, names = ["n1", "n2", "e", "w"])
df

I am not sure if it is really necessary to create the matrices A and w, if we use the networkx library. w might be usefull, but maybe A is not needed. I just tried it out before discovering the networkx package...
The code like this gives an (n-1)x(n-1) data frame where only the upper triangel is filled. If we want to represent it differently (e.g. symmetric matrix, boolean matrix, 2Dnumpy array, real upper triangular matrix etc) more transformation is needed.

I think for w it makes sense to use the matrix, because we only need it for calculating f. We have to be carefull with index, because the matrix has dimension (n-1)x(n-1) because the diagonal of the n x n matrix would just be 0 (connecting each node with itself costs nothing). Better to work with the row and column index then...

In [ ]:
# create adjacency matrix as data frame of upper triangular matrix
A = df.pivot(index='n1', columns='n2', values='e') #gives upper triangular data frame
A

In [ ]:
# create weight matrix as data frame of upper triangular matrix
w = df.pivot(index='n1', columns='n2', values='w') #gives upper triangular data frame
w

In [ ]:
w.loc[2,3] # weight of edge connecting node 2 and 3

In [ ]:
np.nansum(A.loc[4,])+np.nansum(A.loc[:,4]) # this should give us d(node4), as it is an upper triangular symmetric matrix. 
# But I don't know if we need this, as it is probably implemented in networkx.

In [ ]:
# try out how networkx works
# create empty graph
g = nx.Graph()

# get currently used edges (plus their weight, if we want)
current_e = df.loc[df["e"]==1][["n1", "n2", "w"]].to_numpy()
# fill graph by loading it with edges (with weights)
g.add_weighted_edges_from(current_e, weight = "w")


In [ ]:
# we can find out which nodes are currently connected
# should give us a list of nodes that are connected
d = list(nx.connected_components(g))
# d contains disconnected subgraphs
# d[0] contains the biggest subgraph
d

In [ ]:
# try this function with a completely disconnected graph to see the difference
g_disconnected = nx.Graph()
g_disconnected.add_nodes_from(range(1, 20))
d1 = list(nx.connected_components(g_disconnected))
d1

In [ ]:
# try looping through components. Will need more testing
for i in d1[0]:
    rem = df.loc[((df["n1"]==i) | (df["n2"]==i)) & (df["e"]==1)][["w"]].sum()
    print(rem)